In [10]:
!pip3 install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to c:\users\kevin\appdata\local\temp\pip-req-build-l3m1vkno
  Resolved https://github.com/huggingface/transformers to commit bdb9106f247fca48a71eb384be25dbbd29b065a8
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9128910 sha256=25f7b0bfbc674421ed61f88259fcde41f349162c1d1bf0e565237cae96239298
  Stored in directory: C:\Users\kevin\AppData\Local\Temp\pip-ephem-wheel-cache-7wt3txy9\wheels\04\a3\f1\b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built 

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\kevin\AppData\Local\Temp\pip-req-build-l3m1vkno'


In [11]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

processed_output_file = "../data/processed/processed_personas_text.txt"

with open(processed_output_file, 'r', encoding='utf-8') as f:
    processed_text = f.read()

max_length = 512
text_chunks = [processed_text[i:i+max_length] for i in range(0, len(processed_text), max_length)]

dataset = Dataset.from_dict({"text": text_chunks})

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=max_length)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()


C:\Users\kevin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\kevin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\lib\shm.dll" or one of its dependencies.